# RAG Without API (Local Models Only)
Complete RAG pipeline using only local/open-source models

In [ ]:
import sys
sys.path.append('../src')

from retrieval import RetrieverSystem
from sentence_transformers import SentenceTransformer

## Step 1: Load Retriever

In [ ]:
# Load retriever
retriever = RetrieverSystem.load_index(
    '../index/faiss.index',
    '../index/embeddings.npy',
    '../index/corpus_chunks.json',
    '../index/corpus_meta.json'
)

# Load embedding model
model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
print("✅ Models loaded")

## Step 2: Simple Retrieval-Only System

In [ ]:
def simple_rag(query, k=3):
    """
    Simple RAG without LLM:
    1. Retrieve relevant chunks
    2. Return chunks as answer
    """
    # Retrieve
    query_emb = model.encode([query])[0]
    results = retriever.search(query_emb, k=k)
    
    return {
        'query': query,
        'results': results
    }

## Step 3: Test Retrieval System

In [ ]:
# Test queries
test_queries = [
    "كيف أحصل على رخصة ليموزين؟",
    "ما هي خطوات تسجيل المقررات في جامعة قطر؟",
    "كيف أطلب استشارة طبية؟",
    "ما هي إجراءات تقديم العروض للمناقصات؟",
]

for query in test_queries:
    print(f"\n{'='*80}")
    print(f"السؤال: {query}")
    print('='*80)
    
    result = simple_rag(query)
    
    print(f"\nالنتائج المسترجعة:")
    for r in result['results']:
        print(f"\n[{r['rank']}] Score: {r['score']:.3f}")
        print(f"الفئة: {r['metadata']['category']}")
        print(f"الملف: {r['metadata']['source_file'].split('/')[-1]}")
        print(f"المحتوى: {r['chunk'][:300]}...")

## Step 4: Template-Based Answer Generation

In [ ]:
def template_answer(query, k=3):
    """
    Generate structured answer using templates (no LLM)
    """
    query_emb = model.encode([query])[0]
    results = retriever.search(query_emb, k=k)
    
    # Build answer from top result
    top_result = results[0]
    
    answer = f"""بناءً على المعلومات المتاحة:

{top_result['chunk']}

المصدر: {top_result['metadata']['source_file'].split('/')[-1]}
الفئة: {top_result['metadata']['category']}
درجة التطابق: {top_result['score']:.2%}
"""
    
    return answer

In [ ]:
# Test template-based answers
for query in test_queries[:3]:
    print(f"\n{'='*80}")
    print(f"السؤال: {query}")
    print('='*80)
    print(template_answer(query))

## Step 5: Interactive Search

In [ ]:
# Interactive mode
print("بحث في الخدمات الحكومية - اكتب 'خروج' للإنهاء\n")

while True:
    query = input("\nسؤالك: ")
    
    if query.strip() in ['خروج', 'exit', 'quit']:
        print("شكراً لاستخدامك نظام البحث!")
        break
    
    if not query.strip():
        continue
    
    print(template_answer(query))

## ✅ RAG System Without API

Features:
- Semantic search with embeddings
- FAISS for fast retrieval
- Template-based answers (no API needed)
- Completely local and free

Limitations:
- No natural language generation
- Returns raw chunks instead of synthesized answers
- Less conversational

For better answers, use the Gemini version (04_rag_with_gemini.ipynb)